# Big Bird
@hyyoka



self attention은 계산의 특성 상 O(n^2), 즉 문장의 길이에 큰 영향을 받으므로 길이 제한을 둔다. 어떻게 하면 입력 문장의 길이를 성능의 저하없이 늘릴 수 있을까에 대한 연구의 한 결과가 BigBird다. Bigbird는 **graph sparsification**의 아이디어를 적용해 기존 모델들보다 8배 더 긴 sequence input을 다룰 수 있다. 

Three main contributions of **Big Bird** :

1. **Big Bird** satisfies allthe known theoretical properties of full transformer, and is Turing Complete.
2. **Big Bird** benefits variety of NLP tasks, including QA and document summarization.
3. **Big Bird** improves performance on downstream tasks such as promoter region and chromatin profile prediction.






In [1]:
# BigBird tokenizer is relying on senetencepiece so we needa install it first

!pip install datasets
!pip install git+https://github.com/vasudevgupta7/transformers.git@add_big_bird
!pip install sentencepiece

     |████████████████████████████████| 235kB 9.5MB/s 
     |████████████████████████████████| 245kB 14.3MB/s 
     |████████████████████████████████| 112kB 17.9MB/s 
  Cloning https://github.com/vasudevgupta7/transformers.git (to revision add_big_bird) to /tmp/pip-req-build-chi9p2ms
  Running command git clone -q https://github.com/vasudevgupta7/transformers.git /tmp/pip-req-build-chi9p2ms
  Running command git checkout -b add_big_bird --track origin/add_big_bird
  Switched to a new branch 'add_big_bird'
  Branch 'add_big_bird' set up to track remote branch 'add_big_bird' from 'origin'.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3MB 8.7MB/s 
     |████████████████████████████████| 901kB 33.7MB/s 
  Created wheel for transformers: filename=transformers-4.5.0.dev0-cp37-none-any.whl size=2018594 sha256=adc06029d94e5495affd6e0d3f87dc48c131ffecc65c90bdcfd5e92cd9b4f81

### Dataset

- [`narative-qa dataset`](https://huggingface.co/datasets/narrativeqa_manual): abstractive question answering


In [2]:
!git clone https://github.com/deepmind/narrativeqa --branch master && sh narrativeqa/download_stories.sh

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
339a7a65cc21054c3b86fcc1b089b96ded1db7e5
narrativeqa/download_stories.sh: 37: [: gutenberg: unexpected operator
narrativeqa/download_stories.sh: 43: narrativeqa/download_stories.sh: file: not found
narrativeqa/download_stories.sh: 44: narrativeqa/download_stories.sh: [[: not found
33beeecef8e727e7cf85f29acb4db7e3cc683c95
narrativeqa/download_stories.sh: 37: [: gutenberg: unexpected operator
narrativeqa/download_stories.sh: 43: narrativeqa/download_stories.sh: file: not found
narrativeqa/download_stories.sh: 44: narrativeqa/download_stories.sh: [[: not found
33d7b5c04cc6220def312952b53737f1173a5936
narrativeqa/download_stories.sh: 37: [: gutenberg: unexpected operator
narrativeqa/download_stories.sh: 43: narrativeqa/download_stories.sh: file: not found
narrativeqa/download_stories.sh: 44: narrativeqa/download_stories.sh: [[: not found
33e2da2b7d2b57cad0fa960b31444c14afeb2056
narrativeqa/download_stories.sh: 37: [: gutenberg: unexpected operator
narrat

In [6]:
from datasets import load_dataset

In [7]:
dataset = load_dataset("narrativeqa_manual", data_dir="narrativeqa/tmp")
dataset

Using custom data configuration default-data_dir=narrativeqa%2Ftmp


Dataset narrativeqa_manual downloaded and prepared to /root/.cache/huggingface/datasets/narrativeqa_manual/default-data_dir=narrativeqa%2Ftmp/1.0.0/c48529f57560e11e1a7dbb88d3ec6fa0150d25d8f56b71b856cd6a2ee2111b2f. Subsequent calls will reuse this data.


DatasetDict({
    train: Dataset({
        features: ['document', 'question', 'answers'],
        num_rows: 32747
    })
    test: Dataset({
        features: ['document', 'question', 'answers'],
        num_rows: 10557
    })
    validation: Dataset({
        features: ['document', 'question', 'answers'],
        num_rows: 3461
    })
})

In [8]:
tr_dataset = dataset["train"]
val_dataset = dataset["validation"]
tr_dataset, val_dataset

(Dataset({
     features: ['document', 'question', 'answers'],
     num_rows: 32747
 }), Dataset({
     features: ['document', 'question', 'answers'],
     num_rows: 3461
 }))

In [9]:
data = dataset["train"].map(lambda x: {"seqlen": (len(x["document"]["summary"]["text"]) + len(x["question"]))//4})
data = data.map(lambda x: {"q_seqlen": len(x["question"]["text"])//4})

In [10]:
# lets decide whether we should use BigBird `block_sparse` attention or `original_full` attention
min(data["seqlen"]), sum(data["seqlen"])/len(data["seqlen"]), max(data["seqlen"])

# Since avg seqlen < 1024, we should use `original_full` but for the purpose of demonstarting `block_spare` lets try `block_sparse` attention only

(282, 848.2507710629982, 1511)

In [11]:
# lets see the seqlen of question. This may give some idea about the value of `block_size`
min(data["q_seqlen"]), sum(data["q_seqlen"])/len(data["q_seqlen"]), max(data["q_seqlen"])

# lets take `block_size=64`

(2, 11.46517238220295, 43)

### Training BigBird



In [12]:
from transformers import EncoderDecoderModel, BigBirdModel, BigBirdForCausalLM, BigBirdTokenizer
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

encoder = BigBirdModel.from_pretrained("google/bigbird-roberta-base", block_size=64, num_random_blocks=3, attention_type="block_sparse")
decoder = BigBirdForCausalLM.from_pretrained("google/bigbird-roberta-base", add_cross_attention=True, is_decoder=True)
model = EncoderDecoderModel(encoder=encoder, decoder=decoder)

tokenizer = BigBirdTokenizer.from_pretrained("google/bigbird-roberta-base")

When using `BigBirdForCausalLM` as decoder, then `attention_type` must be `original_full`. Setting `attention_type=original_full`
Some weights of the model checkpoint at google/bigbird-roberta-base were not used when initializing BigBirdForCausalLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BigBirdForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BigBirdForCausalLM were not initialized from the model checkpoint at google/bigbird-roberta-base and are newly initialized: ['bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.

In [13]:
SRC_MAXLEN = 832
TGT_MAXLEN = 32

In [14]:
# Function for tokenization

def collate_fn(features):

  context = [x["document"]["summary"]["text"] for x in features]
  question = [x["question"]["text"] for x in features]
  answer = [x["answers"][0]["text"] for x in features]

  # should not eliminate special tokens since question and context are should have `SEP` in middle
  inputs = tokenizer(question, context, return_tensors="pt", padding="max_length", truncation=True, max_length=SRC_MAXLEN)
  labels = tokenizer(answer, return_tensors="pt", padding=True, truncation=True, max_length=TGT_MAXLEN)

  return {
      "input_ids": inputs.input_ids,
      "attention_mask": inputs.attention_mask,
      "decoder_input_ids": labels.input_ids,
      "labels": labels.input_ids,
      "decoder_attention_mask": labels.attention_mask,
  }

In [16]:
args = Seq2SeqTrainingArguments(
    output_dir="bigbird2bigbird-narrative-qa",
    overwrite_output_dir=False,
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
    per_device_train_batch_size=1,       # initial setting was 4
    per_device_eval_batch_size=1,        # initial setting was 4
    gradient_accumulation_steps=2,
    learning_rate=5e-5,
    num_train_epochs=10,
    logging_strategy="steps",
    logging_steps=4000,
    save_strategy="epoch",
    run_name="bigbird2bigbird-narrative-qa-experiment1",
    load_best_model_at_end=True,
)

In [17]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    data_collator=collate_fn,
    train_dataset=tr_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()